In [16]:
import pandas as pd
import json
import re

In [17]:
def extract_info(text):
    pattern = r"\[(?!(Variable))(\w+.*?)\]"
    matches = re.findall(pattern, str(text))
    
    result = {}
    for match in matches:
        parts = match[1].split(':')
        if len(parts) == 2:
            result[parts[0].strip()] = parts[1].strip().strip("'")
    
    return result

In [18]:
def replace_pattern(json_string):
    pattern = r"\[(?!(Variable))(\w+.*?)\]"
    
    # Find all matches in the JSON string
    matches = re.findall(pattern, json_string)
    
    # Iterate through matches and replace in JSON string
    for match in matches:
        key = match[1].split(':')[0].strip()
        json_string = json_string.replace(f'[{match[1]}]', f'{{{key}}}')
    
    return json_string

In [33]:
df = pd.read_excel('docs\Prompts-doc_1.xlsx')
df2 = pd.read_excel('docs\Prompts-doc_2.xlsx')

excel_df = df.fillna('')
excel_df2 = df2.fillna('')

prompt_system = "You are kheops, a superintelligent artificial intelligence developed by Kheops AI team, and your purpose and drive is to assist the user with any request they have."

l = []
for i, title in enumerate(excel_df["Titres"]):
    m = {}
    inputs = extract_info(excel_df["Prompts"][i])
    prompt_user = ''

    if excel_df['Prompts'][i] != '':
        prompt_user = excel_df['Prompts'][i]

    excel_df['Prompts'][i] = replace_pattern(excel_df['Prompts'][i])
    
    m[title] = {'prompt_system' : prompt_system, 'prompt_user' : excel_df["Prompts"][i], 'inputs' : inputs}
    l.append(m)

for i, title in enumerate(excel_df2["Titres"]):
    m = {}
    inputs = extract_info(excel_df2["Prompts"][i])
    prompt_user = ''

    if excel_df2['Prompts'][i] != '':
        prompt_user = excel_df2['Prompts'][i]

    excel_df2['Prompts'][i] = replace_pattern(excel_df2['Prompts'][i])
    
    m[title] = {'prompt_system' : prompt_system, 'prompt_user' : excel_df2["Prompts"][i], 'inputs' : inputs}
    l.append(m)

merged_map = {}

for item in l:
    for key, value in item.items():
        if key in merged_map:
            merged_map[key].append(value)
        else:
            merged_map[key] = [value]


In [34]:
with open('formatted_data_merged.json', 'w', encoding='utf-8') as file:
    json.dump(merged_map, file, indent=2, ensure_ascii=False)